In [1]:
import numpy as np
import torch

from train import EGBADTrainer
from preprocess import get_mnist

In [2]:
class Args:
    num_epochs=100
    lr=1e-4
    latent_dim=256
    anormal_class=1
    batch_size=128
    pretrained=False
    
    
args = Args()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = get_mnist(args)

egbad = EGBADTrainer(args, data, device)

In [3]:
egbad.train()

60000/60000: [===============================>] - ETA 0.4sss
Training... Epoch: 0, Discrimiantor Loss: 0.031, Generator Loss: 17.806
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 1, Discrimiantor Loss: 0.001, Generator Loss: 26.728
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 2, Discrimiantor Loss: 0.001, Generator Loss: 30.853
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 3, Discrimiantor Loss: 0.000, Generator Loss: 34.406
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 4, Discrimiantor Loss: 0.001, Generator Loss: 36.387
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 5, Discrimiantor Loss: 0.001, Generator Loss: 40.797
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 6, Discrimiantor Loss: 0.001, Generator Loss: 42.852
60000/60000: [===============================>] - ETA 0.0ss
Training... Epo

60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 62, Discrimiantor Loss: 0.279, Generator Loss: 11.311
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 63, Discrimiantor Loss: 0.290, Generator Loss: 10.808
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 64, Discrimiantor Loss: 0.330, Generator Loss: 10.445
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 65, Discrimiantor Loss: 0.311, Generator Loss: 10.226
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 66, Discrimiantor Loss: 0.337, Generator Loss: 9.878
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 67, Discrimiantor Loss: 0.355, Generator Loss: 9.495
60000/60000: [===============================>] - ETA 0.0ss
Training... Epoch: 68, Discrimiantor Loss: 0.380, Generator Loss: 9.198
60000/60000: [===============================>] - ETA 0.0ss
Training... 

KeyboardInterrupt: 

In [ ]:
from torch.autograd import Variable
import torch.nn.functional as F


alpha = 0.9
labels = []
scores = []

egbad.G.eval()
egbad.D.eval()

with torch.no_grad():
    for img, label in data[1]:
        img = img.float().to(device)
        img_hat = egbad.G(egbad.E(img))
        score_g = torch.sum(torch.abs(img - img_hat), dim=(1,2,3))
        
        y_true = Variable(torch.ones((img.size(0), 1)).to(device))
        y_pred = egbad.D(img, egbad.E(img)).view(-1)
        score_d = F.binary_cross_entropy(y_pred, y_true)
        
        score = alpha * score_g + (1-alpha)* score_d
        scores.append(score.cpu())
        labels.append(label.cpu())

In [ ]:
scores = torch.cat(scores, dim=0)
labels = torch.cat(labels, dim=0)

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc

precision, recall, thresholds = precision_recall_curve(labels, scores)

print('ROC AUC score: {:.2f}'.format(roc_auc_score(labels, scores)*100))
print('PR AUC score: {:.2f}'.format(auc(recall, precision)*100))

In [ ]:
import matplotlib.pyplot as plt

plt.hist(scores[labels==0], color='r', alpha = 0.3)
plt.hist(scores[labels==1], color='b', alpha=0.3)
plt.show()